In [ ]:
import openai
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base = os.getenv

from pprint import pprint
import pandas as pd

In [ ]:
import requests
url = requests.get(url='https://www.volza.com/p/soyabean/import/import-in-india/')

In [ ]:
from bs4 import BeautifulSoup as bs
soup = bs(url.text,'lxml')

In [ ]:
import json
res = soup.find_all('span',attrs='ant-table-column-title')

In [ ]:
res

In [ ]:
columns = []
for i in res[:5]:
    columns.append(''.join(i.text.upper()))

In [ ]:
data = pd.DataFrame(columns=columns)

In [ ]:
data

In [ ]:
for i in soup.find_all('tr'):
    row_data = i.find_all('td')
    row = [j.text for j in row_data[:5]]
    length = len(data)
    data.loc[length] = row




In [ ]:
data.info()

## From Saudi Aramco website

In [ ]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.aramco.com/en/news-media/news-archive'
get = requests.get(url)

In [ ]:
soup = BeautifulSoup(requests.get(url).content)

In [ ]:
for i in soup.find_all('h3',attrs='search-results__result-title'):
    print(i.text)

In [ ]:
url = 'https://www.aramco.com/en/news-media/news-archive?query=&facets=content_item_type_s%3dNews&sort=pubdate%7cTrue&pubstart=&pubend=&page='
def aramco_scraper(url:str,pages:int=68):
     cont = []
     for i in range(0,pages+1):
         url2 = url + str(i)
         response = requests.get(url2)
         soup = BeautifulSoup(response.content, 'lxml')
         results = soup.find_all('h3', attrs='search-results__result-title')
         for i in results:
              cont.append(i.text)
     return cont

In [ ]:
news = aramco_scraper(url='https://www.aramco.com/en/news-media/news-archive?query=&facets=content_item_type_s%3dNews&sort=pubdate%7cTrue&pubstart=&pubend=&page=')

In [ ]:
len(news)

In [ ]:
with open('aramco_news.txt','w+') as file:
    for i in news:
        file.write(i+'//\n\n')

scraping the descriptions

In [ ]:
def aramco_scraper(url:str,pages:int=68):
     cont = []
     for i in range(0,pages+1):
         url2 = url + str(i)
         response = requests.get(url2)
         soup = BeautifulSoup(response.content, 'lxml')
         results = soup.find_all('p', attrs='search-results__result-description')
         for i in results:
              cont.append(i.text)
     return cont

In [ ]:
desc = aramco_scraper(url=url)

In [ ]:
with open('aramco_news_desc.txt', 'w+') as f:
    for i in desc[1:]:
        f.write(i+'//\n\n')

print('Done!')

### Extracting the news from the links

In [ ]:
from bs4 import  BeautifulSoup
import requests
import json
import time
import os
import re

link = 'https://www.aramco.com/en/news-media/news-archive?query=&facets=content_item_type_s%3dNews&sort=pubdate%7cTrue&pubstart=&pubend=&page='

get = requests.get(link)
soup = BeautifulSoup(get.text,'lxml')

In [ ]:


links = []
for page in range(0,69):
    url = link+str(page)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    for i in soup.find_all('div',attrs='search-results__result-wrapper'):
        for j in i.find_all('a'):
            #The links have facet which directs to the page displayed so we will have to 
            if 'facet' not in j.get('href'):
                links.append('https://www.aramco.com'+j.get('href'))

In [ ]:
len(links)

In [ ]:
links[1]

In [43]:
full_text = []

for i in links:
    url = requests.get(str(i))
    soup =  BeautifulSoup(url.text, 'html.parser')
    for link in soup.find_all('div',attrs='text'):
        para_ = []
        for para in link.find_all('p'):
            para_.append(para.text.strip(','))
        para_ = ' '.join(para_)
        full_text.append(para_)


In [44]:
with open("fulltext.txt","w+") as f:
    f.writelines(full_text)